In [ ]:
import math
import os
import matplotlib.pyplot as plt

# -------------- Geometry --------------
class Punto:
    def __init__(self, x, y):
        self.x, self.y = float(x), float(y)
    def __eq__(self, other):
        return isinstance(other, Punto) and self.x == other.x and self.y == other.y
    def __hash__(self):
        return hash((self.x, self.y))

class Segmento:
    def __init__(self, A, B):
        self.A, self.B = A, B

class Triangulo:
    def __init__(self, puntos):
        self.puntos = puntos
        self.lados = [Segmento(puntos[0], puntos[1]),
                      Segmento(puntos[1], puntos[2]),
                      Segmento(puntos[2], puntos[0])]
        self.centro = None
        self.radio = None


# -------------- Problema 2 (Team 2) --------------
def set_circumcircle(tri: Triangulo):
    ax, ay = tri.puntos[0].x, tri.puntos[0].y
    bx, by = tri.puntos[1].x, tri.puntos[1].y
    cx, cy = tri.puntos[2].x, tri.puntos[2].y
    d = 2 * (ax*(by - cy) + bx*(cy - ay) + cx*(ay - by))
    if abs(d) < 1e-12:
        tri.centro, tri.radio = Punto(float('inf'), float('inf')), float('inf')
        return
    ux = ((ax*ax + ay*ay)*(by - cy) + (bx*bx + by*by)*(cy - ay) + (cx*cx + cy*cy)*(ay - by)) / d
    uy = ((ax*ax + ay*ay)*(cx - bx) + (bx*bx + by*by)*(ax - cx) + (cx*cx + cy*cy)*(bx - ax)) / d
    tri.centro = Punto(ux, uy)
    tri.radio = math.hypot(ux - ax, uy - ay)


# -------------- Problema 3 (Team 3) --------------
def point_in_circumcircle(tri: Triangulo, p: Punto):
    if tri.centro is None or tri.radio is None:
        set_circumcircle(tri)
    return math.hypot(p.x - tri.centro.x, p.y - tri.centro.y) <= tri.radio + 1e-9


# -------------- Problema 4 (Team 4) --------------
def compararSegmentos(seg1: Segmento, seg2: Segmento):
    return (seg1.A == seg2.A and seg1.B == seg2.B) or (seg1.A == seg2.B and seg1.B == seg2.A)

def lados_no_compartidos(lista_triangulos):
    general, repetidos = [], []
    for tri in lista_triangulos:
        for seg in tri.lados:
            found = False
            for i in range(len(general)):
                if compararSegmentos(seg, general[i]):
                    repetidos.append(general.pop(i))
                    found = True
                    break
            if not found:
                if not any(compararSegmentos(seg, r) for r in repetidos):
                    general.append(seg)
    return general


# -------------- Problema 1 (Team 1)--------------
def create_big_triangle(puntos):
    min_x = min(puntos, key=lambda p: p.x).x
    max_x = max(puntos, key=lambda p: p.x).x
    min_y = min(puntos, key=lambda p: p.x).y
    max_y = max(puntos, key=lambda p: p.x).y

    dx = max_x - min_x
    dy = max_y - min_y

    dmax = max(dx, dy)

    centro_x = min_x + dx/2
    centro_y = min_y + dy/2

    p1 = Punto(centro_x - 2 * dmax, centro_y - dmax * 2)
    p2 = Punto(centro_x, centro_y + 2 * dmax)
    p3 = Punto(centro_x + 2 * dmax, centro_y - dmax * 2)

    tri = Triangulo([p1, p2, p3])
    set_circumcircle(tri)

    return tri


# -------------- Bowyer-Watson --------------
def bowyer_watson(puntos, output_dir, base_name):
    big = create_big_triangle(puntos)
    triangulos = [big]

    # Subcarpeta de iteraciones
    iter_dir = os.path.join(output_dir, base_name)
    os.makedirs(iter_dir, exist_ok=True)

    # Imagen del triángulo inicial
    plot_triangulation([big.puntos[0], big.puntos[1], big.puntos[2]], [big],
                       f"{base_name} | Triángulo inicial",
                       show_edges=True, show_circles=True,
                       save_path=os.path.join(output_dir, f"{base_name}_inicial.png"))

    for i, p in enumerate(puntos, 1):
        bad = [t for t in triangulos if point_in_circumcircle(t, p)]
        frontera = lados_no_compartidos(bad)
        triangulos = [t for t in triangulos if t not in bad]
        for lado in frontera:
            nuevo = Triangulo([lado.A, lado.B, p])
            set_circumcircle(nuevo)
            triangulos.append(nuevo)

        # Guardar iteración intermedia
        plot_triangulation(puntos, triangulos, f"{base_name} | Iteración {i}",
                           show_edges=True, show_circles=False,
                           save_path=os.path.join(iter_dir, f"iter_{i}.png"))

    big_vertices = set((big.puntos[0], big.puntos[1], big.puntos[2]))
    triangulos = [t for t in triangulos if not any(v in big_vertices for v in t.puntos)]
    return triangulos


# -------------- Cargar Puntos --------------
def load_points(file_path: str):
    with open(file_path, "r") as f:
        lines = [ln.strip() for ln in f if ln.strip()]
    data_lines = lines[1:]
    return [Punto(*ln.split()) for ln in data_lines]


# -------------- Problema 5 (Team 5) --------------
def plot_triangulation(points, triangles, title, show_edges=True, show_circles=False, save_path=None):
    plt.figure()
    # Dibujar aristas
    if show_edges:
        for t in triangles:
            xs = [t.puntos[i].x for i in range(3)] + [t.puntos[0].x]
            ys = [t.puntos[i].y for i in range(3)] + [t.puntos[0].y]
            plt.plot(xs, ys, color="Red")

    if show_circles:
        xs_all = [p.x for p in points]
        ys_all = [p.y for p in points]
        xmin, xmax = min(xs_all), max(xs_all)
        ymin, ymax = min(ys_all), max(ys_all)
        pad_x = (xmax - xmin) * 0.05 if xmax > xmin else 1.0
        pad_y = (ymax - ymin) * 0.05 if ymax > ymin else 1.0
        plt.xlim(xmin - pad_x, xmax + pad_x)
        plt.ylim(ymin - pad_y, ymax + pad_y)
        for t in triangles:
            if t.centro is None or t.radio is None:
                set_circumcircle(t)
            circ = plt.Circle((t.centro.x, t.centro.y), t.radio, fill=False, color="black")
            plt.gca().add_patch(circ)
    else:
        plt.scatter([p.x for p in points], [p.y for p in points])

    plt.gca().set_aspect('equal', adjustable='box')
    plt.title(title)
    plt.grid(True)
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    else:
        plt.show()


# -------------- Ejecutar todas las instancias --------------
files = [
    "puntos-n10.txt",
    "puntos-n11.txt",
    "puntos-n15.txt",
    "puntos-n16.txt",
    "puntos-n20.txt",
    "puntos-n50.txt",
    "puntos-n100.txt",
]

os.makedirs("resultados", exist_ok=True)

for fp in files:
    pts = load_points(fp)
    base_title = f"{os.path.basename(fp).split('.')[0]}"
    output_dir = "resultados"

    tris = bowyer_watson(pts, output_dir, base_title)

    # Listado de triángulos formados
    print(f"\n--- Triángulos para {fp} ---")
    for i, t in enumerate(tris, 1):
        print(f"Triángulo {i}:")
        for lado in t.lados:
            print(f"  Segmento: ({lado.A.x:.2f}, {lado.A.y:.2f}) -> ({lado.B.x:.2f}, {lado.B.y:.2f})")

    # Gráficas finales
    plot_triangulation(pts, tris, f"{base_title} | Triangulación final",
                       show_edges=True, show_circles=False,
                       save_path=os.path.join(output_dir, f"{base_title}_final.png"))

    plot_triangulation(pts, tris, f"{base_title} | Triangulación + círculos",
                       show_edges=True, show_circles=True,
                       save_path=os.path.join(output_dir, f"{base_title}_final_circulos.png"))



--- Triángulos para puntos-n10.txt ---
Triángulo 1:
  Segmento: (-2.42, -8.47) -> (-8.10, 0.90)
  Segmento: (-8.10, 0.90) -> (-7.12, 1.38)
  Segmento: (-7.12, 1.38) -> (-2.42, -8.47)
Triángulo 2:
  Segmento: (4.13, 6.74) -> (5.72, 9.35)
  Segmento: (5.72, 9.35) -> (8.16, 6.04)
  Segmento: (8.16, 6.04) -> (4.13, 6.74)
Triángulo 3:
  Segmento: (6.77, -3.82) -> (5.37, -5.40)
  Segmento: (5.37, -5.40) -> (2.62, 2.08)
  Segmento: (2.62, 2.08) -> (6.77, -3.82)
Triángulo 4:
  Segmento: (5.37, -5.40) -> (-2.42, -8.47)
  Segmento: (-2.42, -8.47) -> (2.62, 2.08)
  Segmento: (2.62, 2.08) -> (5.37, -5.40)
Triángulo 5:
  Segmento: (-2.42, -8.47) -> (-7.12, 1.38)
  Segmento: (-7.12, 1.38) -> (2.62, 2.08)
  Segmento: (2.62, 2.08) -> (-2.42, -8.47)
Triángulo 6:
  Segmento: (4.13, 6.74) -> (8.16, 6.04)
  Segmento: (8.16, 6.04) -> (2.62, 2.08)
  Segmento: (2.62, 2.08) -> (4.13, 6.74)
Triángulo 7:
  Segmento: (8.16, 6.04) -> (6.77, -3.82)
  Segmento: (6.77, -3.82) -> (2.62, 2.08)
  Segmento: (2.62, 2.08